A simple pairwise interaction feature is the product of two features.

In [4]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preproc

In [3]:
# Load the Online News Popularity dataset

df = pd.read_csv('/home/garvit/Downloads/OnlineNewsPopularity/OnlineNewsPopularity.csv', delimiter=', ')

/home/garvit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df.sample(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
16912,http://mashable.com/2013/12/02/turntable-shuts...,402.0,9.0,958.0,0.511135,1.0,0.705085,12.0,4.0,1.0,...,0.062500,1.000000,-0.513542,-1.000000,-0.100000,0.500000,0.500000,0.000000,0.500000,909
28263,http://mashable.com/2014/07/03/lisa-simpson-fo...,189.0,6.0,49.0,0.916667,1.0,0.964286,1.0,1.0,0.0,...,0.333333,0.333333,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.500000,932
34825,http://mashable.com/2014/10/15/the-superhero-m...,84.0,11.0,518.0,0.500000,1.0,0.703008,9.0,0.0,1.0,...,0.100000,0.800000,-0.386310,-1.000000,-0.125000,0.288889,-0.155556,0.211111,0.155556,684
30039,http://mashable.com/2014/07/31/southwest-seawo...,160.0,7.0,212.0,0.706161,1.0,0.872000,8.0,4.0,0.0,...,0.160000,1.000000,-0.202778,-0.250000,-0.155556,0.642857,0.214286,0.142857,0.214286,2700
33077,http://mashable.com/2014/09/19/climate-change-...,111.0,12.0,298.0,0.602076,1.0,0.725389,15.0,2.0,1.0,...,0.136364,0.750000,-0.186111,-0.333333,-0.100000,0.000000,0.000000,0.500000,0.000000,776


In [5]:
# Assume df is a Pandas DataFrame containing the UCI Online News Popularity dataset
df.columns

Index(['url', 'timedelta', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
       'average_token_length', 'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
     

In [8]:
# Select the content-based features as singleton features in the model,
# skipping over the derived features

features =  ['n_tokens_title', 'n_tokens_content',
             'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
             'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
             'average_token_length', 'num_keywords', 'data_channel_is_lifestyle',
             'data_channel_is_entertainment', 'data_channel_is_bus',
             'data_channel_is_socmed', 'data_channel_is_tech',
             'data_channel_is_world']


x = df[features]
y = df[['shares']]

In [9]:
# Create pairwise interaction features, skipping the constant bias term

x2 = preproc.PolynomialFeatures(include_bias=False)
x2 = x2.fit_transform(x)
x2.shape

(39644, 170)

In [11]:
# Create train/test sets for both feature sets

x1_train, x1_test, x2_train, x2_test, y_train, y_test =  train_test_split(x, x2, y, test_size=0.3, 
                                                                          random_state=123)


In [12]:
def evaluate_feature(x_train, x_test, y_train, y_test):    
    """Fit a linear regression model on the training set andscore on the test set"""
    model = linear_model.LinearRegression()
    model.fit(x_train, y_train)
    r_score = model.score(x_test, y_test)
    return (model, r_score)

# Train models and compare score on the two feature sets
(m1, r1) = evaluate_feature(x1_train, x1_test, y_train, y_test)
(m2, r2) = evaluate_feature(x2_train, x2_test, y_train, y_test)
print("R-squared score with singleton features: %0.5f" % r1)
print("R-squared score with singleton features: %0.5f" % r2)

R-squared score with singleton features: 0.00924
R-squared score with singleton features: 0.01133
